In [20]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib tk
plt.style.use('ggplot')

5608/|/ 19%|| 5608/30000 [01:10<05:04, 80.06it/s, D_fake_loss=0.733, D_real_loss=0.641, G_loss=0.68]

In [21]:
class Generator(nn.Module):
    def __init__(self, in_size, hid_size, out_size):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(in_size, hid_size)
        self.map2 = nn.Linear(hid_size, hid_size)
        self.map3 = nn.Linear(hid_size, out_size)
        
    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.sigmoid(self.map2(x))
        return  self.map3(x)
        
class Discriminator(nn.Module):
    def __init__(self, in_size, hid_size, out_size):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(in_size, hid_size)
        self.map2 = nn.Linear(hid_size, hid_size)
        self.map3 = nn.Linear(hid_size, out_size)
        
    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.elu(self.map2(x))
        return F.sigmoid(self.map3(x))         

In [22]:
def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))

def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n) 

In [25]:
d_in_size = 200

d_sampler = get_distribution_sampler(4, 0.5)
g_sampler = get_generator_input_sampler()
G = Generator(
    in_size=1,
    hid_size=50,
    out_size=1
)
D = Discriminator(
    in_size=d_in_size,
    hid_size=50,
    out_size=1
)

criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr=2e-4, betas=(0.9, 0.999))
g_optimizer = optim.Adam(G.parameters(), lr=2e-4, betas=(0.9, 0.999))
    
def train():
    
    dflosses = []
    drlosses = []
    glosses = []
    
    e_bar = tqdm_notebook(range(50000))
    for e in e_bar:
        d_bar = range(1)#tqdm_notebook(range(1), leave=False)
        g_bar = range(1)#tqdm_notebook(range(1), leave=False)
        
        for d_i in d_bar:
            D.zero_grad()
            # Train on real
            d_real_data = Variable(d_sampler(d_in_size))
            d_real_dec = D(d_real_data)
            d_real_err = criterion(d_real_dec, Variable(torch.ones(1,1)))
            d_real_err.backward()
            # Train on fake
            d_gen_in = Variable(g_sampler(d_in_size, 1))
            d_fake_data = G(d_gen_in).detach()
            d_fake_dec = D(d_fake_data.t())
            d_fake_err = criterion(d_fake_dec, Variable(torch.zeros(1,1)))
            d_fake_err.backward()
            d_optimizer.step()
            
            drlosses.append(d_real_err.data[0])
            dflosses.append(d_fake_err.data[0])
            
        
        for g_i in g_bar:
            G.zero_grad()
            
            gen_in = Variable(g_sampler(d_in_size, 1))
            g_fake_data = G(gen_in)
            dg_fake_dec = D(g_fake_data.t())
            g_err = criterion(dg_fake_dec, Variable(torch.ones(1,1)))
            g_err.backward()
            g_optimizer.step()
            
            glosses.append(g_err.data[0])
            
        e_bar.set_postfix(
            D_real_loss=drlosses[-1],
            D_fake_loss=dflosses[-1],
            G_loss=glosses[-1]
        )
    return drlosses, dflosses, glosses
   

In [26]:
drl, dfl, gl = train()

4198/|/ 14%|| 4198/30000 [00:53<05:29, 78.32it/s, D_fake_loss=0.751, D_real_loss=0.721, G_loss=0.653]

In [27]:
plt.plot(drl, label='real')
plt.plot(dfl, label='fake')
plt.plot(gl, label='gen')
plt.legend()

In [36]:
gen_in = Variable(g_sampler(10000, 1))
gg = G(gen_in)

In [37]:
plt.hist(gg.data.numpy(), bins=20)
plt.show()
print(np.std(gg.data.numpy()))
print(np.mean(gg.data.numpy()))

0.441987
4.30014


In [ ]:
plt.hist(d_sampler(100).numpy().flatten(), bins=20)
plt.show()